In [1]:
import pandas as pd
import requests
import json
import datetime
import time
from dateutil.relativedelta import relativedelta
import math

import sqlalchemy
from sqlalchemy.types import Integer, Text, String, Float, Date
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
database_name = "../data/CompanyData.sqlite"
engine = create_engine(f"sqlite:///{database_name}", echo=False)
master_df = pd.read_sql_query("SELECT * FROM MasterData", engine)

In [3]:
# Create a list of the possible criteria
criteria_list = ["price_earnings", "price_book", "market_cap", "net_debt_capital", "ev_revenue", "ev_ebit"]
# criteria_list = ["price_earnings", "ev_ebit"]

# Create a dataframe of the sector names
sector_df = pd.read_sql_query("SELECT DISTINCT sector FROM MasterData", engine)

# Append the sector "All"
sector_df = sector_df.append([{"sector":"All"}], ignore_index=True)

# Create a dataframe of all the monthend dates
date_df = pd.read_sql_query("SELECT DISTINCT monthend_date FROM MasterData", engine)

# Drop dates 2014-10-31 and 2014-11-30, as there is not enough company data for analysis
date_df = date_df[date_df.monthend_date != '2014-10-31']
date_df = date_df[date_df.monthend_date != '2014-11-30']



In [4]:
# Create an empty dataframe to hold the results
quintile_returns_df = pd.DataFrame()

for test_sector in sector_df['sector']:
    print ('Working on', test_sector)
    for criteria in criteria_list:
        print ('    ', criteria)
        for test_date in date_df['monthend_date']:
            
            # Start with a fresh copy of the master_df
            test_df = master_df
            # Narrow down for the specific "monthend_date"
            test_df = test_df.loc[test_df['monthend_date'] == test_date]
            # Narrow down for the specific "test_sector"
            if (test_sector != "All"):
                test_df = test_df.loc[test_df['sector'] == test_sector]
            # Copy the item to be tested to a new column column called "criteria"
            # Note that PE, PB, EV/Revenue and EV/EBIT are tested on the reciprocal value
            # It has to do with the behavior of the ratio when the denominator gets close to zero
            if (criteria == "price_earnings"):
                test_df['criteria'] = test_df['earnings_yield']
            elif (criteria == "price_book"):
                test_df['criteria'] = test_df['book_yield']
            elif (criteria == "market_cap"):
                test_df['criteria'] = test_df['market_cap']
            elif (criteria == "net_debt_capital"):
                test_df['criteria'] = test_df['net_debt_capital']
            elif (criteria == "ev_revenue"):
                test_df['criteria'] = test_df['revenue_ev']
            elif (criteria == "ev_ebit"):
                test_df['criteria'] = test_df['ebit_ev']

            # Perform the quintile calculation on the criteria, store the result in a new "quintile" column
            # and calculate the average return for all stocks in each quintile
            # Financial stocks don't have EBIT.  So when this combination shows up, set the "avg_returns" to zero
            if ((criteria == "ev_ebit") and (test_sector == "Financials")):
                avg_returns = [0,0,0,0,0,0]
            else:
                # Calculate the quintile for each stock
                # Quants count quintiles from 1 to 5, not 0 to 4, so add 1 to the python formula
                # QCUT will set the lowest values to quintile 1, the highest values to quintile 5
                # That works for all criteria, except for 'market_cap', where we want high values to be 5, low to be 1
                if (criteria == "market_cap"):
                    test_df['quintile'] = 5 - pd.qcut(test_df['criteria'], 5, labels=False)
                else:
                    test_df['quintile'] = pd.qcut(test_df['criteria'], 5, labels=False) + 1

                # Calculate the average total return for all stocks in each quintile
                avg_returns = test_df.groupby('quintile')['total_return'].mean()

            # Store the results in the quintile_returns_df
            for i in range(1,6):
                one_dictionary = {"monthend_date":test_date, "sector":test_sector, "criteria": criteria, "quintile": i, "port_return": avg_returns[i]}
                one_row_df = pd.DataFrame([one_dictionary])
                quintile_returns_df = quintile_returns_df.append (one_row_df, sort=False, ignore_index=True)

Working on Health Care
     price_earnings
     price_book
     market_cap
     net_debt_capital
     ev_revenue
     ev_ebit
Working on Industrials
     price_earnings
     price_book
     market_cap
     net_debt_capital
     ev_revenue
     ev_ebit
Working on Consumer Discretionary
     price_earnings
     price_book
     market_cap
     net_debt_capital
     ev_revenue
     ev_ebit
Working on Information Technology
     price_earnings
     price_book
     market_cap
     net_debt_capital
     ev_revenue
     ev_ebit
Working on Consumer Staples
     price_earnings
     price_book
     market_cap
     net_debt_capital
     ev_revenue
     ev_ebit
Working on Utilities
     price_earnings
     price_book
     market_cap
     net_debt_capital
     ev_revenue
     ev_ebit
Working on Financials
     price_earnings
     price_book
     market_cap
     net_debt_capital
     ev_revenue
     ev_ebit
Working on Real Estate
     price_earnings
     price_book
     market_cap
     net_debt_capit

C:\Users\Brad\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Brad\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     price_book


C:\Users\Brad\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     market_cap


...

C:\Users\Brad\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     ev_revenue


C:\Users\Brad\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     ev_ebit


C:\Users\Brad\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
# Sort the dataframe by sector, criteria quintile and monthend ascending date
quintile_returns_df.sort_values(['sector', 'criteria', 'quintile', 'monthend_date'], inplace=True)
quintile_returns_df.head(12)

,criteria,monthend_date,port_return,quintile,sector
20235,ev_ebit,2014-12-31,-0.754444,1,All
20240,ev_ebit,2015-01-31,5.942398,1,All
20245,ev_ebit,2015-02-28,-0.686324,1,All
20250,ev_ebit,2015-03-31,0.945363,1,All
20255,ev_ebit,2015-04-30,2.336419,1,All
20260,ev_ebit,2015-05-31,-1.005397,1,All
20265,ev_ebit,2015-06-30,0.885059,1,All
20270,ev_ebit,2015-07-31,-5.997421,1,All
20275,ev_ebit,2015-08-31,-4.060622,1,All
20280,ev_ebit,2015-09-30,7.728753,1,All


In [6]:
# To calculate the average annual return for each quintile, we have to link the returns sequentially
# That is, the avg annual return = ((1+month_1_return/100) * (1+month_2_return/100) * ... * (1+month_n_return/100)) ^ (12/n)
# where "n" is the number of months.

# Here's the math trick to do that.  Convert (1+month_return/100) to a log number, then 
# take the average of the log numbers and multiply by 12.  Convert back to a return number.

# Calculate the log of each return, store it in a new column called "log_return"
quintile_returns_df["log_return"] = [math.log(1 + x/100) for x in quintile_returns_df["port_return"]]

# Calculate the cumulative sum of the log_return, grouping by ticker
quintile_returns_df["sum_log"] = quintile_returns_df.groupby(['sector', 'criteria', 'quintile'])['log_return'].cumsum()

# Convert the 'sum_log' column by raising e to the power of 'sum_log', then multiplying by 100
quintile_returns_df["wealth_index"] = [(math.exp(x))*100 for x in quintile_returns_df["sum_log"]]

# Drop the "sum_log" columns
quintile_returns_df.drop(columns=["sum_log"], inplace=True)
quintile_returns_df.head()

,criteria,monthend_date,port_return,quintile,sector,log_return,wealth_index
20235,ev_ebit,2014-12-31,-0.754444,1,All,-0.007573,99.245556
20240,ev_ebit,2015-01-31,5.942398,1,All,0.057725,105.143122
20245,ev_ebit,2015-02-28,-0.686324,1,All,-0.006887,104.421499
20250,ev_ebit,2015-03-31,0.945363,1,All,0.009409,105.408661
20255,ev_ebit,2015-04-30,2.336419,1,All,0.023095,107.871449


In [7]:
# Check to make sure we have the expected number of rows in the quintile_returns_df
# 6 Criteria * 57 Months (4 years + 9 months) * 12 sectors * 5 quintiles = 20,520 rows expected
len (quintile_returns_df['monthend_date'])

20520

In [8]:
# To write this dataframe to an SQL table with any keys, do the following
# Use the "df.to_sql" to write the dataframe to an SQL table with a temporary name
final_table_name = "QuintileMonthlyData"

# Now, write the dataframe to the SQL table
quintile_returns_df.to_sql('Temp', con=engine, if_exists='replace', index=False)

# Drop any existing NEW table, the one that will contain the data and index
sql_stmt = "DROP TABLE " + final_table_name
engine.execute(sql_stmt)

# Create a NEW table to hold all the data, and be sure to include the index definition
create_table_sql = 'CREATE TABLE "' + final_table_name + '" (' + \
    'criteria VARCHAR, ' + \
    'monthend_date DATE, ' + \
    'port_return FLOAT, ' + \
    'quintile INT, ' + \
    'sector VARCHAR, ' + \
    'log_returns VARCHAR, ' + \
    'wealth_index VARCHAR, ' + \
    'PRIMARY KEY (monthend_date, sector, criteria, quintile));'

# Create the table.  It will be empty
engine.execute(create_table_sql)

# Now copy the data from the Temp table into the New table
sql_stmt = "INSERT INTO " + final_table_name + " SELECT * FROM Temp"
engine.execute(sql_stmt)

# Delete the Temp table
sql_stmt = "DROP TABLE Temp"
engine.execute(sql_stmt)

In [9]:
# Find the average of the log of returns and mulitply by 12
avg_returns_df = quintile_returns_df.groupby(['sector','criteria','quintile'])['log_return'].mean().reset_index()
avg_returns_df["log_return"] = avg_returns_df["log_return"] * 12

# Now raise e to the power of the log_return, subtract 1 and multiply by 100.
# That results in average annual return
avg_returns_df["avg_return"] = [(math.exp(x)-1)*100 for x in avg_returns_df["log_return"]]

# Drop the "log_return" column
avg_returns_df.drop(columns="log_return", inplace=True)

avg_returns_df.head()

,sector,criteria,quintile,avg_return
0,All,ev_ebit,1,12.473228
1,All,ev_ebit,2,13.932246
2,All,ev_ebit,3,12.355254
3,All,ev_ebit,4,10.554549
4,All,ev_ebit,5,8.925574


In [10]:
# Check to see that we have the expected number of rows
# 6 Criteria * 12 months/year * 5 quintiles = 360 rows expected
len (avg_returns_df['sector'])

360

In [11]:
# To write this dataframe to an SQL table with any keys, do the following
# Use the "df.to_sql" to write the dataframe to an SQL table with a temporary name
final_table_name = "QuintileAvgData"

# Now, write the dataframe to the SQL table
avg_returns_df.to_sql('Temp', con=engine, if_exists='replace', index=False)

# Drop any existing NEW table, the one that will contain the data and index
sql_stmt = "DROP TABLE " + final_table_name
engine.execute(sql_stmt)

# Create a NEW table to hold all the data, and be sure to include the index definition
create_table_sql = 'CREATE TABLE "' + final_table_name + '" (' + \
    'sector VARCHAR, ' + \
    'criteria VARCHAR, ' + \
    'quintile INT, ' + \
    'port_return FLOAT, ' + \
    'PRIMARY KEY (sector, criteria, quintile));'

# Create the table.  It will be empty
engine.execute(create_table_sql)

# Now copy the data from the Temp table into the New table
sql_stmt = "INSERT INTO " + final_table_name + " SELECT * FROM Temp"
engine.execute(sql_stmt)

# Delete the Temp table
sql_stmt = "DROP TABLE Temp"
engine.execute(sql_stmt)

# Q.E.D.